# Imports

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sliding_window import SlidingWindow

# Globals

In [2]:
df = pd.read_csv('full_real_data_new.gz', compression='gzip')
df = df.drop(['Unnamed: 0'], axis=1)

In [3]:
df["partc"] = 1

In [5]:
num_experiments = 18
num_participants = 1
exclude = 0
analytic_functions_list = ['mean', 'sum', 'median', 'min', 'max', 'std']
labels_dict = {'wlk': 0, 'sit': 1, "std": 2, "ups": 3, "jog": 4, "dws": 5}

# Helper Functions

In [6]:
def data_allocation_only_split_feature_labels(df):
    # Define X,Y
    df = df.sample(frac=1).reset_index(drop=True)
    X, y = df.drop(["action"], axis=1), df["action"]
    y = y.replace(labels_dict)
    
    return X, y

In [7]:
from sklearn.metrics import classification_report, confusion_matrix

def create_classes(labels_dict):
    classes_indexs = labels_dict.items()
    classes_indexs = sorted(classes_indexs, key=lambda x: x[1])
    classes_names = [label for label, index in classes_indexs]
    return classes_names

def evaluate_results(y_true, y_pred, classes):
        print("---- Printing classification report ----")
        print(classification_report(y_true, y_pred, target_names=classes))

# Model Evaluation

## Prepare Dataframe to Classify

In [8]:
window_size = 20
window = SlidingWindow(df, window_size, num_experiments, num_participants, exclude, analytic_functions_list)
sld_df = window.df
X_test, y_test = data_allocation_only_split_feature_labels(sld_df)
classes_names = create_classes(labels_dict)

## Import trained Random Forest Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
rf = joblib.load('rf_trained.pkl')

### Evaluate Results

In [13]:
prediction = rf.predict(X_test)
evaluate_results(y_test, prediction, classes_names)

---- Printing classification report ----
             precision    recall  f1-score   support

        wlk       0.77      0.35      0.48     52612
        sit       0.97      0.70      0.81     35195
        std       0.76      0.97      0.85     36531
        ups       0.37      0.79      0.50     21760
        jog       0.00      0.00      0.00         0
        dws       0.40      0.46      0.42     19507

avg / total       0.71      0.63      0.63    165605



[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.1s finished
C:\Users\oklainfeld\AppData\Local\Continuum\Anaconda2_new\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
